## Baseline EDA

This notebook takes the predictions on an initial training of the ELECTRA model on the SQuAD dataset (epochs=4) and performs baseline exploration of the predictions.

In [12]:
import json
import os
from pprint import pprint
from typing import Dict

import pandas as pd

In [57]:
BASE_DIR = "/Users/marktorres/Documents/school/ut_austin/2022_fall/nlp/final_project/fp-dataset-artifacts/" # noqa
pd.set_option('display.max_colwidth', None)
pd.set_option('display.max_columns', None)
pd.set_option('display.max_rows', 10)
pd.set_option('display.width', None)

In [7]:
preds_location = os.path.join(BASE_DIR, "data", "eval_predictions.jsonl")

res_list = []

with open(preds_location, 'r') as f:
  json_list = list(f)

for json_str in json_list:
  res_list.append(json.loads(json_str))

In [8]:
def is_answer_exact_match(pred: Dict) -> bool:
  """Evaluates if a prediction is within the set of possible answer choices.

  Allows us to evaluate "exact match" metric for ELECTRA evaluation. The
  "exact match" that we see during the model's evaluation portion should
  be replicated with this function.
  
  Returns boolean.
  """

  possible_answers = pred["answers"]["text"]
  predicted_answer = pred["predicted_answer"]
  return predicted_answer in possible_answers

### Splitting and processing the dataset

As a first step, let's split our data between the ones that the model got correct and the ones that the model did not get correct, in order to establish some baselines.

In [17]:
is_correct_bool_list = [is_answer_exact_match(res) for res in res_list]

In [16]:
preds_df = pd.DataFrame(res_list)

In [18]:
preds_df["is_exact_match_bool"] = is_correct_bool_list

As a first pass, let's see what the breakdown of correct/incorrect is based on the document used for context:

In [44]:
num_rows_by_title_and_correctness_df = pd.crosstab(index=preds_df["title"], columns=preds_df["is_exact_match_bool"], margins=True)
num_rows_by_title_and_correctness_df['proportion_correct'] = num_rows_by_title_and_correctness_df[True]/num_rows_by_title_and_correctness_df['All']*100


In [46]:
num_rows_by_title_and_correctness_df.sort_values(by="proportion_correct")

is_exact_match_bool,False,True,All,proportion_correct
title,,,,
Packet_switching,48,58,106,54.716981
Civil_disobedience,83,114,197,57.868020
Martin_Luther,173,306,479,63.883090
Computational_complexity_theory,70,127,197,64.467005
Pharmacy,43,86,129,66.666667
Chloroplast,97,198,295,67.118644
Intergovernmental_Panel_on_Climate_Change,34,70,104,67.307692
Economic_inequality,95,202,297,68.013468
European_Union_law,76,167,243,68.724280


There's some pattern in which documents the model did not successfully train on. Let's investigate these in order to see what it could be about those samples that could explain why the model does so poorly on them.

In [50]:
def return_df_subset(df: pd.DataFrame, column: str, query: str) -> pd.DataFrame:
    """Return a subset of the df based on a certain column value."""
    return df.loc[df[column] == query]

In [51]:
packet_switching_rows = return_df_subset(preds_df, "title", "Packet_switching")

In [58]:
packet_switching_rows.loc[packet_switching_rows["is_exact_match_bool"] == False]

,id,title,context,question,answers,predicted_answer,is_exact_match_bool
4809,5725d34089a1e219009abf52,Packet_switching,"Starting in the late 1950s, American computer scientist Paul Baran developed the concept Distributed Adaptive Message Block Switching with the goal to provide a fault-tolerant, efficient routing method for telecommunication messages as part of a research program at the RAND Corporation, funded by the US Department of Defense. This concept contrasted and contradicted the theretofore established principles of pre-allocation of network bandwidth, largely fortified by the development of telecommunications in the Bell System. The new concept found little resonance among network implementers until the independent work of Donald Davies at the National Physical Laboratory (United Kingdom) (NPL) in the late 1960s. Davies is credited with coining the modern name packet switching and inspiring numerous packet switching networks in Europe in the decade following, including the incorporation of the concept in the early ARPANET in the United States.",What did this concept contradict,"{'text': ['This concept contrasted and contradicted the theretofore established principles of pre-allocation of network bandwidth', 'theretofore established principles of pre-allocation of network bandwidth', 'principles of pre-allocation of network bandwidth'], 'answer_start': [328, 373, 397]}",established principles of pre-allocation of network bandwidth,False
4815,5725d52f89a1e219009abf79,Packet_switching,"Packet switching contrasts with another principal networking paradigm, circuit switching, a method which pre-allocates dedicated network bandwidth specifically for each communication session, each having a constant bit rate and latency between nodes. In cases of billable services, such as cellular communication services, circuit switching is characterized by a fee per unit of connection time, even when no data is transferred, while packet switching may be characterized by a fee per unit of information transmitted, such as characters, packets, or messages.",What is circuit switching characterized by,"{'text': ['circuit switching is characterized by a fee per unit of connection time', 'a method which pre-allocates dedicated network bandwidth specifically for each communication session', 'fee per unit of connection time'], 'answer_start': [323, 90, 363]}",a fee per unit of connection time,False
4816,5725d52f89a1e219009abf7a,Packet_switching,"Packet switching contrasts with another principal networking paradigm, circuit switching, a method which pre-allocates dedicated network bandwidth specifically for each communication session, each having a constant bit rate and latency between nodes. In cases of billable services, such as cellular communication services, circuit switching is characterized by a fee per unit of connection time, even when no data is transferred, while packet switching may be characterized by a fee per unit of information transmitted, such as characters, packets, or messages.",How is packet switching charecterized,"{'text': ['by a fee per unit of information transmitted', 'a fee per unit of information transmitted', 'fee per unit of information transmitted'], 'answer_start': [474, 477, 479]}","packet switching may be characterized by a fee per unit of information transmitted, such as characters, packets, or messages.",False
4819,572634a789a1e219009ac56e,Packet_switching,"Packet switching contrasts with another principal networking paradigm, circuit switching, a method which pre-allocates dedicated network bandwidth specifically for each communication session, each having a constant bit rate and latency between nodes. In cases of billable services, such as cellular communication services, circuit switching is characterized by a fee per unit of connection time, even when no data is transferred, while packet switching may be characterized by a fee per unit of information transmitted, such as characters, packets, or messages.",H

Is there a difference in the length of the average answer?